### Preprocess arff data and save it in npy format

In [ ]:
%load_ext autoreload  # Load the extension
%autoreload 2  # Autoreload all modules

In [1]:
import os
import time
import shutil

from scipy.io import arff
from scipy import interpolate
import numpy as np

In [2]:
# Only save equirectanguar monoscopic video data
VIDEO_NAMES = [
    '01_art_gallery_static.arff',
    '02_theatre_scene_static.arff',
    '03_lions_static_cuts.arff',
    '04_copenhagen_harbour_static_cuts.arff',
    '05_florida_yacht_motion.arff',
    #'06_dance_experience_static.arff',
    '07_factory_robots_static.arff',
    '08_madagascar_cuts.arff',
    '09_elephants_static.arff',
    #'10_cruise_festival_skate_cuts.arff',
    '11_lohdi_garden_india_cuts.arff',
    '12_gym_workout_static.arff',
    '13_times_square_static.arff',
    '14_new_orleans_drive_motion.arff',
    #'15_interrogation_static.arff',
    '16_interview_static.arff',
    '17_weather_forecast_static.arff',
    '18_bomb_trapped_static.arff',
    #'19_nurse_motion.arff',
    '20_car_fix_static.arff',
    '21_operation_room_static.arff',
    #'22_surgical_checklist_static.arff',
    '23_dog_food_factory_motion.arff',
    #'24_nurse_motion_challenge.arff',
    #'25_theatre_scene_static_challenge.arff',
    #'26_new_orleans_drive_motion_challenge.arff',
    #'27_times_square_static_challenge.arff',
    #'28_dance_experience_static_challenge.arff'
]

In [3]:
COLUMNS = ['x', 'y', 'x_head', 'y_head', 'angle_deg_head', 'quaternion_hp_w',
           'quaternion_hp_x', 'quaternion_hp_y', 'quaternion_hp_z', 'frameId']

def is_outlier(x_1, x_2, x_3, threshold):
    return (abs(x_1 - x_2) >= threshold >= abs(x_1 - x_3)
            and abs(x_2 - x_3) >= threshold)

In [4]:
CURRENT_DIR = os.getcwd()
DATASET_PATH = os.path.join(CURRENT_DIR, 'datasetTUM')
# DELTA_T = 10 milliseconds
DELTA_T = 10000 # microseconds

START_TIME = time.time()
print("------------------------")
print("Importing data... \n")

# TODO: EDIT RANGES
shutil.rmtree(os.path.join(CURRENT_DIR, 'preprocessed_train'))
shutil.rmtree(os.path.join(CURRENT_DIR, 'preprocessed_test'))

os.mkdir('preprocessed_train')
os.mkdir('preprocessed_test')
count_closed_eyes = 0
SPLIT_SUBJECT_ID = 41
# Loop through every user from 2 to 50
for i in range(2, 50+1):
    subjectId = str(i).zfill(3)
    # Loop through every video
    for j, video_name in enumerate(VIDEO_NAMES):
        filename = subjectId + '_' + video_name
        #subjectId = "016"
        #filename = "016_13_times_square_static.arff"
        fname = os.path.join(DATASET_PATH, subjectId, filename)
        print(fname)
        try:
            # Load arff to structured array + metadata
            data, meta = arff.loadarff(fname)
        except FileNotFoundError:
            print("File " + filename + " does not exist.")
            continue

        # Deal with closed eyes in the beginning or end of video sequence
        if data['confidence'][0] == 0:
            count_closed_eyes += 1
            print("User closed eyes! (" + str(count_closed_eyes) + ")")
            continue

        final_index = len(data) - 1
        while data['confidence'][final_index] != 1:
            final_index -= 1

        initial_index = 0
        while data['confidence'][initial_index] != 1:
            initial_index += 1

        data = data[initial_index:final_index+1]

        print(data['x'][0], data['y'][0], data['x_head'][0], data['y_head'][0])

        data_temp = data.copy()
        outlier_threshold = 500

        for column in ['x', 'y', 'x_head', 'y_head']:
            for k in range(len(data)-2):
                if is_outlier(data[column][k], data[column][k+1], data[column][k+2],
                              outlier_threshold):
                    print("catched")
                    data_temp[column][k+1] = (data[column][k] + data[column][k+2]) / 2

        data = data_temp

        t_new = np.arange(data['time'][0], data['time'][-1], DELTA_T)

        input_data_video = np.zeros((len(t_new), len(COLUMNS)))
        for k, column in enumerate(COLUMNS):
            if column in ['x', 'y']:
                index_confidence_0 = (data['confidence'] == 0)
                data_filtered = data[np.logical_not(index_confidence_0)]
            else:
                data_filtered = data

            f = interpolate.interp1d(data_filtered['time'], data_filtered[column])
            input_data_video[:, k] = f(t_new)

        if i <= SPLIT_SUBJECT_ID:
            save_dir = "preprocessed_train"
        else:
            save_dir = "preprocessed_test"

        npy_name = os.path.join(save_dir, filename[:-5]+".npy")
        np.save(npy_name, input_data_video)
        print("Saved file", npy_name)

print("\n\nData import finished.")
print("Importation time:", time.time() - START_TIME, "s,",
      (time.time() - START_TIME) / 60, "min",
      (time.time() - START_TIME) / 3600, "h")
print("------------------------")

------------------------
Importing data... 

/media/demo/DATA/saliency-exploitation/sw/datasetTUM/002/002_01_art_gallery_static.arff
1920.0 1080.0 2027.99 872.8
catched
Saved file preprocessed_train/002_01_art_gallery_static.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/002/002_02_theatre_scene_static.arff
1920.0 960.0 1900.13 946.19
catched
catched
catched
catched
catched
Saved file preprocessed_train/002_02_theatre_scene_static.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/002/002_03_lions_static_cuts.arff
1920.0 960.0 1877.68 934.91
Saved file preprocessed_train/002_03_lions_static_cuts.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/002/002_04_copenhagen_harbour_static_cuts.arff
1920.0 1080.0 1907.0 1062.74
catched
catched
catched
Saved file preprocessed_train/002_04_copenhagen_harbour_static_cuts.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/002/002_05_florida_yacht_motion.arff
User closed eyes! (1)
/media/demo/DATA/saliency-exploitation/s

Saved file preprocessed_train/004_08_madagascar_cuts.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/004/004_09_elephants_static.arff
1920.0 1024.0 1884.16 996.93
catched
catched
catched
catched
catched
catched
catched
catched
catched
Saved file preprocessed_train/004_09_elephants_static.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/004/004_11_lohdi_garden_india_cuts.arff
1920.0 1080.0 1889.64 1045.76
catched
catched
catched
catched
catched
catched
catched
catched
catched
catched
catched
catched
catched
Saved file preprocessed_train/004_11_lohdi_garden_india_cuts.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/004/004_12_gym_workout_static.arff
1920.0 1024.0 1905.31 974.15
catched
catched
catched
catched
catched
catched
Saved file preprocessed_train/004_12_gym_workout_static.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/004/004_13_times_square_static.arff
1920.0 960.0 1895.47 911.29
catched
Saved file preprocessed_train/004_13_times_square_static

Saved file preprocessed_train/006_11_lohdi_garden_india_cuts.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/006/006_12_gym_workout_static.arff
1920.0 1024.0 1927.56 969.4
catched
catched
catched
catched
catched
catched
Saved file preprocessed_train/006_12_gym_workout_static.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/006/006_13_times_square_static.arff
1920.0 960.0 1950.02 941.58
catched
catched
catched
catched
catched
Saved file preprocessed_train/006_13_times_square_static.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/006/006_14_new_orleans_drive_motion.arff
1920.0 1080.0 1894.68 1004.14
catched
catched
Saved file preprocessed_train/006_14_new_orleans_drive_motion.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/006/006_16_interview_static.arff
1920.0 1024.0 1891.77 1088.26
Saved file preprocessed_train/006_16_interview_static.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/006/006_17_weather_forecast_static.arff
1920.0 960.0 1892.98

1914.42 994.43 1944.08 937.05
catched
catched
catched
Saved file preprocessed_train/008_17_weather_forecast_static.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/008/008_18_bomb_trapped_static.arff
User closed eyes! (7)
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/008/008_20_car_fix_static.arff
1920.0 960.0 1876.35 931.91
catched
catched
catched
catched
catched
Saved file preprocessed_train/008_20_car_fix_static.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/008/008_21_operation_room_static.arff
1920.0 960.0 1809.46 914.83
catched
catched
catched
catched
catched
catched
catched
catched
Saved file preprocessed_train/008_21_operation_room_static.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/008/008_23_dog_food_factory_motion.arff
1927.1 1145.63 1889.01 1027.13
catched
catched
Saved file preprocessed_train/008_23_dog_food_factory_motion.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/009/009_01_art_gallery_static.arff
1920.0 1080.0 1923.34 1

1920.0 960.0 1904.11 905.02
catched
catched
catched
catched
catched
catched
Saved file preprocessed_train/011_02_theatre_scene_static.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/011/011_03_lions_static_cuts.arff
1920.0 960.0 1907.05 901.4
catched
catched
catched
Saved file preprocessed_train/011_03_lions_static_cuts.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/011/011_04_copenhagen_harbour_static_cuts.arff
1920.0 1080.0 1887.35 1062.85
catched
catched
catched
catched
catched
Saved file preprocessed_train/011_04_copenhagen_harbour_static_cuts.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/011/011_05_florida_yacht_motion.arff
1920.0 1024.0 1920.28 991.7
catched
Saved file preprocessed_train/011_05_florida_yacht_motion.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/011/011_07_factory_robots_static.arff
1920.0 960.0 1841.33 943.6
catched
catched
catched
catched
catched
Saved file preprocessed_train/011_07_factory_robots_static.npy
/media/demo/DA

...

1920.0 960.0 1866.72 937.63
catched
catched
catched
catched
Saved file preprocessed_train/040_07_factory_robots_static.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/040/040_08_madagascar_cuts.arff
1920.0 1024.0 1895.46 1101.1
catched
catched
catched
Saved file preprocessed_train/040_08_madagascar_cuts.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/040/040_09_elephants_static.arff
1920.0 1024.0 1888.21 1086.41
Saved file preprocessed_train/040_09_elephants_static.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/040/040_11_lohdi_garden_india_cuts.arff
1920.0 1080.0 1855.87 1186.41
catched
catched
catched
catched
Saved file preprocessed_train/040_11_lohdi_garden_india_cuts.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/040/040_12_gym_workout_static.arff
1920.0 1024.0 1909.97 1133.46
catched
catched
Saved file preprocessed_train/040_12_gym_workout_static.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/040/040_13_times_square_static.arff
1920.

1918.24 1008.9 1924.37 1087.36
catched
catched
catched
catched
catched
catched
catched
catched
catched
catched
catched
catched
catched
catched
Saved file preprocessed_test/042_16_interview_static.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/042/042_17_weather_forecast_static.arff
1920.0 960.0 1962.85 1069.65
catched
catched
catched
catched
catched
catched
catched
catched
catched
catched
catched
catched
catched
catched
catched
catched
catched
catched
catched
catched
Saved file preprocessed_test/042_17_weather_forecast_static.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/042/042_18_bomb_trapped_static.arff
1920.0 1080.0 1956.68 1261.98
catched
catched
catched
Saved file preprocessed_test/042_18_bomb_trapped_static.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/042/042_20_car_fix_static.arff
1920.0 960.0 1937.92 998.51
catched
catched
catched
catched
catched
catched
catched
catched
catched
catched
catched
catched
catched
Saved file preprocessed_test/042_20

Saved file preprocessed_test/045_01_art_gallery_static.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/045/045_02_theatre_scene_static.arff
1920.0 960.0 1777.8 927.85
catched
catched
catched
catched
catched
catched
catched
catched
catched
Saved file preprocessed_test/045_02_theatre_scene_static.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/045/045_03_lions_static_cuts.arff
1920.0 960.0 1957.09 919.55
catched
catched
catched
Saved file preprocessed_test/045_03_lions_static_cuts.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/045/045_04_copenhagen_harbour_static_cuts.arff
1920.0 1080.0 2177.87 1206.92
catched
catched
catched
catched
catched
Saved file preprocessed_test/045_04_copenhagen_harbour_static_cuts.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/045/045_05_florida_yacht_motion.arff
1920.0 1024.0 1881.2 1024.72
catched
catched
catched
catched
Saved file preprocessed_test/045_05_florida_yacht_motion.npy
/media/demo/DATA/saliency-exploitation/sw

1920.0 1024.0 1901.33 996.43
Saved file preprocessed_test/047_09_elephants_static.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/047/047_11_lohdi_garden_india_cuts.arff
1920.0 1080.0 1925.06 1088.41
catched
catched
catched
Saved file preprocessed_test/047_11_lohdi_garden_india_cuts.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/047/047_12_gym_workout_static.arff
1920.0 1024.0 1928.71 1016.03
catched
catched
Saved file preprocessed_test/047_12_gym_workout_static.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/047/047_13_times_square_static.arff
1920.0 960.0 1911.54 953.76
catched
catched
catched
catched
catched
catched
catched
catched
catched
catched
Saved file preprocessed_test/047_13_times_square_static.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/047/047_14_new_orleans_drive_motion.arff
1920.0 1080.0 1926.09 1058.96
catched
catched
Saved file preprocessed_test/047_14_new_orleans_drive_motion.npy
/media/demo/DATA/saliency-exploitation/sw/datase

1920.0 960.0 1939.66 908.54
catched
catched
catched
catched
catched
catched
catched
catched
Saved file preprocessed_test/049_17_weather_forecast_static.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/049/049_18_bomb_trapped_static.arff
1920.0 1080.0 1902.2 1072.39
catched
catched
catched
catched
catched
catched
catched
Saved file preprocessed_test/049_18_bomb_trapped_static.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/049/049_20_car_fix_static.arff
1920.0 960.0 1895.38 927.29
Saved file preprocessed_test/049_20_car_fix_static.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/049/049_21_operation_room_static.arff
1920.0 960.0 1920.63 953.32
catched
catched
Saved file preprocessed_test/049_21_operation_room_static.npy
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/049/049_23_dog_food_factory_motion.arff
1920.0 1080.0 1784.27 1088.07
catched
catched
catched
catched
catched
catched
catched
catched
catched
catched
Saved file preprocessed_test/049_23_dog_foo